In [1]:
import pandas as pd

In [2]:
import os

In [3]:
ratings = pd.read_csv(r'D:\Netology\Pandas\ml-latest-small\ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movies = pd.read_csv(r'D:\Netology\Pandas\ml-latest-small\movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
## объединение 2-х файлов
ID_and_title = ratings.merge(movies, on='movieId', how='left')
ID_and_title.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [6]:
## функция классификации
def classification (row) :
    if row ['rating'] <= 2 :
        return "Низкий рейтинг"
    if row ['rating'] <= 4 :
        return "Средний рейтинг"
    return "Высокий рейтинг"

In [7]:
ID_and_title['class'] = ID_and_title.apply(classification, axis = 1)

In [8]:
## последние 20 строк чтобы увидеть 3 варанта рейтинга
ID_and_title.tail(5)

,userId,movieId,rating,timestamp,title,genres,class
99999,671,6268,2.5,1065579370,Raising Victor Vargas (2002),Comedy|Drama|Romance,Средний рейтинг
100000,671,6269,4.0,1065149201,Stevie (2002),Documentary,Средний рейтинг
100001,671,6365,4.0,1070940363,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX,Средний рейтинг
100002,671,6385,2.5,1070979663,Whale Rider (2002),Drama,Средний рейтинг
100003,671,6565,3.5,1074784724,Seabiscuit (2003),Drama,Средний рейтинг


In [9]:
Keywords = pd.read_csv(r'D:\Netology\Pandas\ml-latest-small\keywords.csv')
Keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [10]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
geo_data

{'Центр': ['москва', 'тула', 'ярославль'],
 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [18]:
# функция для поиска городов в запросах
def search_reg1 (row):
    for reg, cities in geo_data.items():
        for citi in cities:
            if citi in row:
                 return reg
    return 'undefined'

In [19]:
Keywords['region'] = Keywords['keyword'].apply(search_reg1)
Keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [20]:
# вызов строк с городом Псков
Keywords[Keywords['keyword'].str.contains('псков')].head()

,keyword,shows,region
2153,авито псков,98931,Северо-Запад
5642,погода псков,47395,Северо-Запад
11222,псков,24885,Северо-Запад
16094,плн псков,18735,Северо-Запад
17360,гисметео псков,17500,Северо-Запад


In [11]:
# создание df из словаря
geo_data_pd = pd.DataFrame.from_dict(geo_data, orient ='index').stack().reset_index()
del geo_data_pd['level_1']
geo_data_pd.columns = ['Region','keyword']
geo_data_pd

,Region,keyword
0,Центр,москва
1,Центр,тула
2,Центр,ярославль
3,Северо-Запад,петербург
4,Северо-Запад,псков
5,Северо-Запад,мурманск
6,Дальний Восток,владивосток
7,Дальний Восток,сахалин
8,Дальний Восток,хабаровск


In [16]:
# объединение двух df Keywords и geo_data_pd и замена не найденных соответствий NaN на 'undefined'
key_and_region = Keywords.merge(geo_data_pd, on='keyword', how='left')
key_and_region.fillna('undefined').head()

,keyword,shows,Region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [14]:
# пример что по keyword москва подставлено Region Центр
key_and_region.loc[key_and_region['keyword'] == 'москва']

,keyword,shows,Region
1259,москва,153050,Центр


In [24]:
key_and_region = Keywords.merge(geo_data_pd, on='keyword', how='inner')
key_and_region.fillna('undefined').head()

,keyword,shows,Region
0,москва,153050,Центр
1,владивосток,43038,Дальний Восток
2,ярославль,41720,Центр
3,мурманск,29984,Северо-Запад
4,хабаровск,29655,Дальний Восток
